In [ ]:
!pip install pymorphy2
!pip install stop_words
!pip install annoy
!pip install python-telegram-bot==12.8
!pip install dialogflow


In [ ]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import regex as re
import random

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/for_colab")

Mounted at /content/for_colab


In [10]:
!unzip "/content/for_colab/My Drive/for_colab/Geekbrains.zip" -d /content
!mv /content/Geekbrains/Otvety.txt /content/


Archive:  /content/for_colab/My Drive/for_colab/Geekbrains.zip
  inflating: /content/Geekbrains/Otvety.txt  


In [12]:
# assert False

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [13]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [14]:
def preprocess_sentence(w):
    spls = re.sub(r'<[^>]*>','', w)
    spls = re.sub(r'[\xa0]',' ', spls)
    spls = re.sub(r'.{1,2} [\n]','', spls)
    spls = re.sub(r'[\n]','', spls)
    spls = re.sub(r'\s[.]\s','', spls)
    spls = re.sub(r"\)\)\)\)", "))", spls)
    spls = re.sub(r"\)\)", ")", spls)
    spls = re.sub(r"\) \)", "))", spls)
    spls = re.sub(r',{2,}','', spls)
    return spls

In [15]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_sentence(line)
        spls = preprocess_txt(spls)
        sentences.append(spls)
        c += 1
        if c > 1900000:
            break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


In [16]:
sentences = [i for i in sentences if len(i) > 2]

In [17]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

In [18]:
embedding_size = 300
window_size = 5
min_word = 5
down_sampling = 1e-3

In [19]:
fastText_model = FastText(sentences=sentences,
                          size=embedding_size,
                          window=window_size,
                          min_count=min_word,
                          sample=down_sampling,
                          sg=1,
                          workers=8)
fastText_model.save('out/ft_model.h5')
fastText_model.wv.most_similar(['книга'], topn=10)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('книгаа', 0.932357907295227),
 ('книг»', 0.862906277179718),
 ('книгуя', 0.8385841846466064),
 ('книгу»', 0.8099246621131897),
 ('книгун', 0.8094027638435364),
 ('книгута', 0.796430230140686),
 ('кни', 0.7760363817214966),
 ('книга»', 0.7749806642532349),
 ('книгия', 0.7676824927330017),
 ('«книга', 0.7579026222229004)]

In [ ]:
ft_index = annoy.AnnoyIndex(embedding_size ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_ft = np.zeros(embedding_size)
        for word in question:
            if word in fastText_model:
                vector_ft += fastText_model[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save('out/ft_index')

with open('out/index_map.pkl', 'wb') as f:
    pickle.dump(index_map, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [ ]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(300)
    for word in txt:
        if word in fastText_model:
            vector_ft += fastText_model[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [ ]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import dialogflow
updater = Updater(token='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx') # Токен API к Telegram
dispatcher = updater.dispatcher
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'xxxxxxxxxxxxxxxxxxxxxxxxxx.json'# скачнный JSON

# DIALOGFLOW_PROJECT_ID = 'nlp-gb-bot-ivuw' #PROJECT ID из DialogFlow 
# DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
# SESSION_ID = 'nlp_gb_bot'  # ID бота из телеграма

In [ ]:
# Обработка команд
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Добрый день! Меня зовут Мифо7, мои исходники найдете здесь: \
    https://github.com/rajulun/Introduction_to_Natural_Language_Processing/tree/master/chat-bot')
    # logging.info("Start Bot")
def textMessage(bot, update):

    input_txt = preprocess_txt(update.message.text)
    # vect = vectorizer.transform([" ".join(input_txt)])
    # prediction = lr.predict(vect)
    # response = 'Ваше сообщение принял ' + update.message.text # формируем текст ответа
    # bot.send_message(chat_id=update.message.chat_id, text=response)
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    response = index_map[ft_index_val[0]]
    response = preprocess_sentence(response)

    if distances[0] < 0.2:
        bot.send_message(chat_id=update.message.chat_id, text=response)
        print(f"distances: {distances[0]}")

    if distances[0] > 0.2 and distances[0] < 0.6:
        print(f"distances: {distances[0]}")
        response = 'Если я правильно понял: ' + response
        bot.send_message(chat_id=update.message.chat_id, text = response)
        return
    elif distances[0] > 0.6:
        print(f"distances: {distances[0]}")
        response = "Пытаюсь понять Вас, но мне сложно. Возможно: " + response
        bot.send_message(chat_id=update.message.chat_id, text = response)
        return

    print(f"Пользователь: {update.message.text}")
    print(f"Бот: {response}")


In [ ]:
# Хендлеры

start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспатчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота на Ctrl + C
updater.idle()

In [ ]:
# !cp out/ft_model.h5 /content/for_colab/My\ Drive/Chat-bot